# Curso Análisis y limpieza de datos

**Profesor responsable:** Jorge Alexis Castillo Sepúlveda, PhD, MSc & Math. Eng.

## Capitulo 1: Limpieza de datos

### Clase 1: Obtención , organización y manejo de datos usando la librería Pandas de Python

En esta clase veremos como obtener datos y cierto manejo básico en la librería Pandas de Python de modo de poder inferir que tipo de datos estamos trabajando, usando más de un caso de estudio.

### Dataset clásicos

En el ámbito de la *ciencia de datos*, es usual hablar de *datsets clásicos*. Estos datasets generalmente vienen incluídos en paquetes y han servido como ejemplos prácticos a la hora de aprender métodos en esta disciplina, a lo largo de varias generaciones de estudiantes. Nosotros no seremos la excepción y nos servirán para explorar los datos y aprender a usar los comandos básicos.



### Datasets no clásicos

Por otra parte, no siempre tendremos a nuestra disposición datasets clásicos. Es más, en las aplicaciones de verdad, ya sea en la academia o en la industria, tendremos que acostumbrarnos a usar datasets "sucios", o que provienen de fuentes extrañas, o que no estén en un formato deseado. El primer paso de un *data scientist* es lidiar con ese tipo de datasets y es bueno que empecemos a acostumbrarnos a trabajar con ellos. 

### Importando un csv (comma separated values) desde una carpeta específica 

In [2]:
import os
os.chdir('C:/Users/aleph/Dropbox/Docencia posgrado/Analisis de Datos  MDS19/datasets')

"""
Dentro del paréntesis deben poner la URL de la carpeta donde guardaron el archivo IRIS.csv
"""

In [3]:
#importando la libreria pandas
import pandas as pd # despues del "as" puedes poner LO QUE QUIERAS, pero por convención, se usa pd

In [4]:
data=pd.read_csv('IRIS.csv')

#generalmente se usa df, y de nuevo, tu le puedes poner el nombre que quieras ;)

In [ ]:
# viendo el encabezado de la data para ver como es

data.head() #dentro del parentesis puedes poner el numero entero que quieras 

In [ ]:
# ¿que tipos de datos tiene nuestro dataframe?
data.dtypes

In [ ]:
# ¿como se mueven las variables continuas?
data.describe()

In [ ]:
# ¿y qué pasa con la columna "species"?

#veamos las categorías que contiene:
data.species.unique()

# el método unique sirve para  mostrar las posibilidades únicas de cada columna. Su equivalente en R es unique(...)

#### Observación: Python es un lenguaje muy intuitivo y es posible usar los "métodos" para denominar a las columnas. Pero para que esto sea posible, el string o label de la columna debe ser una "unidad" completa o monomio. Si tiene un nombre que exceda el "monomio", se recomienda usar el guión bajo para generar el monomio.

#### Suponga que un investigador a cargo del área donde usted trabaja le dice que sólo nos interesa analizar la especie iris versicolor, ya que esa especie tiene recomendaciones para problemas hepáticos.

Vamos a crear un dataframe donde solo esté esa especie.


In [ ]:
#En pandas, para filtrar se debe usar el comando lógico "==" 

df_aux=data[data.species=='Iris-virginica']

In [ ]:
df_aux

In [ ]:
#Describiendo los datos

df_aux.describe()

#### Ejercicio: inferencias sobre los datos:

- Si un colega que está trabajando con los mismos datos, le dice que vio en el dataset una Iris-virginica con pétalo de largo 4.845 . ¿Usted le creería?
- Si alguien dice que ha visto Iris-virginica con pétalos de largo 7, ¿qué le diría?


#### Para indagar:

- Lea la documentación correspondiente a los métodos **describe** y **unique** de la librería pandas y aplique algo distinto al paréntesis vacío. 

### Importando un csv desde una url


In [ ]:
#### importando desde una URL

data2=pd.read_csv('https://bit.ly/2ZNXHTQ')


In [ ]:
data2.head(10)

In [ ]:
data2.describe()

In [ ]:
data2.dtypes

In [ ]:
#cada columna representa una serie
type(data2.año)

Parece ser que tenemos un datset que es una serie de tiempo, en donde las columnas que tienen que ver con la fecha, están separadas y no se ve un orden aparente. Hay otras dos coliumnas que corresponden a cantidad y venta, es decir, estamos hablando de un dataset que proviene directamente de la industria.

#### Creando la columna de fechas

Lo que hay que hacer, es crear una columuna uniendo lo que se ve en la columna de año, en la columna de mes y en la columna de día. ¿Cómo lo hacemos? Hay más de una manera. 

### Método 1: Concatenar

In [ ]:
#esto lanzará un error

data2['fecha']=data2.dia+ '-' + data2.mes+'-'+data2.año

**A tener en cuenta** : Tener errores no nos hace menos data scientists ni nada por el estilo. No existe ser humano que no haya tenido errores en programación. Los errores sirven para aprender, es la gracia de la investigación y la ciencia en general. Quien busque la perfección debe replantearse. 

*¿Donde está el error?* En que se quiere concatenar datos que no son string (excepto por el guión "-")

**La solución es usar el método map, cambiando el tipo de dato a string**. Esto es porque cada columna es una Serie (Series) y map actúa sobre cada elemento de una serie.

In [ ]:
data2['fecha']=data2.dia.map(str) + '-' + data2.mes.map(str) + '-' + data2.año.map(str)

In [ ]:
data2['fecha']

La idea entonces es ordenar los datos por fecha. Veamos que pasa si ordeno solo la columna de fechas recién creada.


In [ ]:
data2.sort_values(by=['fecha']) 

**¿qué pasó? ¿es correcto el orden?**

####  Debemos transformar el string a fecha usando pd.to_datetime

In [ ]:
data2['fecha']=pd.to_datetime(data2['fecha'])

In [ ]:
data2.sort_values(by=['fecha'], inplace=True) # se puede usar con inplace=True para no tener que hacer x=x(condicion nueva)

In [ ]:
data2

In [ ]:
# Sólo nos interesa la fecha, la venta y la cantidad, por lo tanto, seleccionamos las columans de interés:

data2=data2[['fecha','cantidad','venta']]
data2


In [ ]:
""""
Por último, observamos que los indices del nuevo dataframe están desordenados. 
Para el análisis de datos, en ocasiones es MUY IMPORTANTE que los índices estén ordenados
Para eso usamos el método reset_index. Debemos usar drop si no queremos que el índice antiguo 
se transforme en una columna nueva
"""""
data2.reset_index(inplace=True, drop=True)
data2

### Método 2: Usando el método  apply, format y el cálculo lambda 

- El método **apply** sirve para aplicar una función a toda una columna o toda una fila en un determinado dataframe en pandas.
- El formateo (format) hace que cada expresion vacía se llene con lo que está dentro del parentesis '{}'. Por lo tanto es una manera *simple* de pasar a string.   
- EL **cálculo lambda** proviene de la lógica matemática para definir el concepto de función y abordar el problema de la recursividad. En python se usa para definir funciones de una manera simple y directa usando la siguiente lógica:

<h3><center>lambda variable: expresion(variable)</center></h3>


Por lo tanto, este método, que es más sotisificado que el anterior, consiste en aplicar a toda la columna un formateo de string usando una función definida por el cálculo lambda. Luego pasamos a formato de fecha y reseteamos el índice. 

In [ ]:
data3=pd.read_csv('https://bit.ly/2ZNXHTQ')

#en una sola línea:
data3['fecha']=data3[['dia','mes','año']].apply(lambda x : '{}-{}-{}'.format(x[0],x[1],x[2]), axis=1)

###

data3['fecha']=pd.to_datetime(data3['fecha'])
data3=data3[['fecha','cantidad','venta']]
data3.sort_values('fecha', inplace=True)
data3.reset_index(inplace=True, drop=True)
data3

### Ejercicios

- Calcular una columna de precio
- El cliente necesita las salidas en el formato YY-MM-DD. Cree otro dataset con las fechas en ese formato usando cualquier de los dos métodos (de preferencia el 2)
- Suponga que un data scientist senior de su empresa le pide analizar **un año de datos** con fecha máxima el mes julio 2019, ya que se debe evaluar las ventas del ultimo año y en producción aun no se cierra el mes de agosto 2019. Cree un dataframe auxiliar con esta consideración.
- ¿Por qué hay cree usted que hay dos datos por cada fecha?

**Tarea**

- Cree un nuevo dataset en donde cada fecha debe aparecer una sola vez.
- Calcule el precio promedio del último año cerrado
- ¿En cuántos días las transacciones estuvieron por sobre los 650 pesos? 